# Setup Code

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# imports
import torch
import numpy as np
import sys
import os
from matplotlib import pyplot as plt

In [ ]:
# select devices
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Bad to go!")
    DEVICE = torch.device("cpu")

Load config from config files.

In [ ]:
# load config
sys.path.append(os.getcwd())
# choose between ship, lego
import configs.ship, configs.lego
# change config file here
config = configs.lego

Load the dataset and show the first one.

In [ ]:
from nerf.data import load_blender
imgs, poses, int_mat = load_blender(config.datadir, device=DEVICE)
img_n, img_h, img_w = imgs.shape[:3]
# visualize
plt.imshow(np.array(imgs[0].to(device="cpu")))
plt.axis("off")
plt.title("demo image")
plt.show()
print("and its pose: ")
print(np.array(poses[0].to(device="cpu")))

In [ ]:
def get_rays(H, W, K, c2w):
    i, j = torch.meshgrid(torch.linspace(0, W-1, W), torch.linspace(0, H-1, H))  # pytorch's meshgrid has indexing='ij'
    i = i.t().to(K.device)
    j = j.t().to(K.device)
    dirs = torch.stack([(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -torch.ones_like(i)], -1)
    print(dirs.shape)
    print(dirs[..., None, :].shape)
    # Rotate ray directions from camera frame to the world frame
    rays_d = torch.sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)  # dot product, equals to: [c2w.dot(dir) for dir in dirs]
    print(rays_d.shape)
    # Translate camera frame's origin to the world frame. It is the origin of all rays.
    rays_o = c2w[:3,-1].expand(rays_d.shape)
    return rays_o, rays_d

o, d = get_rays(img_h, img_w, int_mat, poses[0])
rays = torch.stack([get_rays(img_h, img_w, int_mat, p)])

In [56]:
# test posencode

from nerf_helper import PosEncode

L = 2
x = torch.tensor([[1.1, 1.2, 1.3], [2.1, 2.2, 2.3]])
enc_x = PosEncode(x, L)

print(enc_x)


tensor([[ 0.8912,  0.4536,  0.8085, -0.5885,  0.9320,  0.3624,  0.6755, -0.7374,
          0.9636,  0.2675,  0.5155, -0.8569],
        [ 0.8632, -0.5048, -0.8716, -0.4903,  0.8085, -0.5885, -0.9516, -0.3073,
          0.7457, -0.6663, -0.9937, -0.1122]])
